In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
from scipy import signal

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GroupKFold, GridSearchCV  , StratifiedGroupKFold, cross_validate
from sklearn.metrics import accuracy_score, balanced_accuracy_score, recall_score
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor, MLPClassifier

from random import sample, shuffle
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone
from copy import deepcopy

import matplotlib.pyplot as plt 
import seaborn as sns

In [6]:
#dataset_path_root = "/home/bruno/Academico/Doctorado/Neuro_Fairness/Shu_Dataset/DATASET_6_BAND_POWER_MEAN_ACRROSS_CHANNELS/"
dataset_path_root = "/home/bzorzet/Academico/Doctorado/Neuro_Fairness/Shu_Dataset/DATASET_6_BAND_POWER_MEAN_ACRROSS_CHANNELS/"
participants=["sub-001","sub-002","sub-003","sub-004","sub-005",
              "sub-006","sub-007","sub-008","sub-009","sub-010",
              "sub-011","sub-012","sub-013","sub-014","sub-015",
              "sub-016","sub-017","sub-018","sub-019","sub-020",
              "sub-021","sub-022","sub-023","sub-024","sub-025"]
sessions = ["ses-01","ses-02","ses-03","ses-04","ses-05"]

In [8]:
dataset={}
for participant in participants:
    dataset[participant]={}
    data_path=participant+"_task_motorimagery_eeg_preprocessing_band_power_features_mean.mat"
    data=sio.loadmat(dataset_path_root + data_path)
    for session in sessions:
        dataset[participant][session +'_data_band_power']=data[session +'_data_band_power_mean']
        dataset[participant][session +'_labels_trials']=data[session +'_labels_trials']
    dataset[participant]['bands_freqs']= data['bands_freqs'] 
    dataset[participant]['sfreq']=np.squeeze(data['sfreq'])
    dataset[participant]['age']=np.squeeze(data['age'])
    dataset[participant]['gender']=data['gender'][0]
    dataset[participant]['group_medidator']=data['group_medidator'][0]
    dataset[participant]['id_participant']=data['id_participant'][0]

In [9]:
index_female = []
index_male = []
for participant in participants:
    if dataset[participant]['gender'] == 'M':
        index_male.append(participant)
    elif dataset[participant]['gender'] == 'F':
        index_female.append(participant)
print(f"Participantes hombres: {index_male}")
print(f"Participantes mujeres: {index_female}")

Participantes hombres: ['sub-001', 'sub-002', 'sub-008', 'sub-012', 'sub-013', 'sub-015', 'sub-017', 'sub-018', 'sub-019', 'sub-021', 'sub-022', 'sub-023', 'sub-025']
Participantes mujeres: ['sub-003', 'sub-004', 'sub-005', 'sub-006', 'sub-007', 'sub-009', 'sub-010', 'sub-011', 'sub-014', 'sub-016', 'sub-020', 'sub-024']


In [11]:
for participant in participants:
    data_ = np.concatenate((dataset[participant]['ses-01_data_band_power'],
                            dataset[participant]['ses-02_data_band_power'],
                            dataset[participant]['ses-03_data_band_power'],
                            dataset[participant]['ses-04_data_band_power'],
                            dataset[participant]['ses-05_data_band_power']),axis=0)
    
    dataset[participant]['data_band_power'] = data_
    dataset[participant]['data_gender'] =  np.array(list(dataset[participant]['gender']) * data_.shape[0])
    dataset[participant]['group_participant'] =  np.array(list([participant]) * data_.shape[0])

In [21]:
n_features = data_.shape[-1]
N_it = 20
n_test_participant = 2
n_val_participant = 2 
n_ign_participant = 1

max_iter = 200
patience = 50 
info_exp = {}
info_clfs = {}
for it in range(N_it):
    dic_aux = {}
    
    X_train = None
    X_val = None
    X_test = None 
    
    X_train_ = None
    X_val_ = None
    X_test_ = None
    
    idx_male = index_male.copy()
    idx_female = index_female.copy()
    
    # TEST PARTICIPANTS:
    idx_male_test = sample(idx_male, n_test_participant)
    idx_female_test = sample(idx_female, n_test_participant)
    for it_ in range(n_test_participant):
        idx_male.remove(idx_male_test[it_])
        idx_female.remove(idx_female_test[it_])
    idx_test = idx_male_test + idx_female_test
    dic_aux['reg_idx_test'] = idx_test   
    
    # VALIDATION PARTICIPANTS:
    idx_male_val = sample(idx_male, n_val_participant)
    idx_female_val = sample(idx_female, n_val_participant)
    for it_ in range(n_val_participant):
        idx_male.remove(idx_male_val[it_])
        idx_female.remove(idx_female_val[it_])
    idx_val = idx_male_val + idx_female_val
    dic_aux['reg_idx_val'] = idx_val  
    
    # TRAIN PARTICIPANTS:
    idx_male_ignore = sample(idx_male, n_ign_participant)
    for it_ in range(n_ign_participant):
        idx_male.remove(idx_male_ignore[it_])
    idx_male_train = idx_male.copy()
    idx_female_train = idx_female.copy()
    idx_train = idx_male_train + idx_female_train
    dic_aux['reg_idx_train'] = idx_train    
        
    
    # CONCATENAMOS EL CONJUNTO DE DATOS
    # TEST
    X_test = np.zeros((1,n_features))
    Y_test = np.zeros(1)
    for participant in idx_test:
        X_test = np.concatenate((X_test, dataset[participant]['data_band_power']),axis=0)
        Y_test = np.concatenate((Y_test, dataset[participant]['data_gender']),axis=0)
    X_test = X_test[1:,:]
    Y_test = Y_test[1:]
    dic_aux['n_trials_test'] = {'male':np.sum(Y_test == 'M'),'female':np.sum(Y_test == 'F')}
    dic_aux['proportion_trials_test'] = {'male':np.sum(Y_test == 'M')/(np.sum(Y_test == 'M')+np.sum(Y_test == 'F')),
                                      'female':np.sum(Y_test == 'F')/(np.sum(Y_test == 'M')+np.sum(Y_test == 'F'))}
    # VALIDATION
    X_val = np.zeros((1,n_features))
    Y_val = np.zeros(1)
    for participant in idx_val:
        X_val = np.concatenate((X_val, dataset[participant]['data_band_power']),axis=0)
        Y_val = np.concatenate((Y_val, dataset[participant]['data_gender']),axis=0)
    X_val = X_val[1:,:]
    Y_val = Y_val[1:]
    dic_aux['n_trials_val'] = {'male':np.sum(Y_val == 'M'),'female':np.sum(Y_val == 'F')}
    dic_aux['proportion_trials_val'] = {'male':np.sum(Y_val == 'M')/(np.sum(Y_val == 'M')+np.sum(Y_val == 'F')),
                                      'female':np.sum(Y_val == 'F')/(np.sum(Y_val == 'M')+np.sum(Y_val == 'F'))}
        
    # TRAIN
    X_train = np.zeros((1,n_features))
    Y_train = np.zeros(1)
    for participant in idx_train:
        X_train = np.concatenate((X_train, dataset[participant]['data_band_power']),axis=0)
        Y_train = np.concatenate((Y_train, dataset[participant]['data_gender']),axis=0)
    X_train = X_train[1:,:]
    Y_train = Y_train[1:]
    dic_aux['n_trials_train'] = {'male':np.sum(Y_train == 'M'),'female':np.sum(Y_train == 'F')}
    dic_aux['proportion_trials_train'] = {'male':np.sum(Y_train == 'M')/(np.sum(Y_train == 'M')+np.sum(Y_train == 'F')),
                                      'female':np.sum(Y_train == 'F')/(np.sum(Y_train == 'M')+np.sum(Y_train == 'F'))}   
    
    info_exp[f'it_{it}']=dic_aux
    dic_aux={}
    
    #-----------------------CLASSIFIER 1 -----------------------------#
    acc_train = None
    acc_val = None
    acc_test = None
    
    tol = 1e-4
    scaler = StandardScaler()
    clf = LinearDiscriminantAnalysis(tol=tol)
    
    X_train_ = scaler.fit_transform(X_train, Y_train)
    X_val_ = scaler.transform(X_val)
    X_test_ = scaler.transform(X_test)
    
    
    clf.fit(X_train_, Y_train)
    acc_train = clf.score(X_train_ , Y_train)
    acc_val = clf.score(X_val_ , Y_val)
    acc_test = clf.score(X_test_, Y_test)
    
    dic_aux['clf_1']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test}
    
    #-----------------------CLASSIFIER 2 -----------------------------#
    acc_train = None
    acc_val = None
    acc_test = None
    
    tol = 1e-4
    scaler = StandardScaler()
    clf = svm.SVC(C=1.0, kernel='linear', tol=tol)
    
    X_train_ = scaler.fit_transform(X_train, Y_train)
    X_val_ = scaler.transform(X_val)
    X_test_ = scaler.transform(X_test)
    
    
    clf.fit(X_train_, Y_train)
    acc_train = clf.score(X_train_ , Y_train)
    acc_val = clf.score(X_val_ , Y_val)
    acc_test = clf.score(X_test_, Y_test)
    
    dic_aux['clf_2']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test}
    
    #-----------------------CLASSIFIER 3 -----------------------------#
    acc_train = None
    acc_val = None
    acc_test = None
    
    tol = 1e-4
    scaler = StandardScaler()
    clf = svm.SVC(C=1.0, kernel='rbf', tol=tol)
    
    X_train_ = scaler.fit_transform(X_train, Y_train)
    X_val_ = scaler.transform(X_val)
    X_test_ = scaler.transform(X_test)
    
    
    clf.fit(X_train_, Y_train)
    acc_train = clf.score(X_train_ , Y_train)
    acc_val = clf.score(X_val_ , Y_val)
    acc_test = clf.score(X_test_, Y_test)
    
    dic_aux['clf_3']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test}
    
    
    #-----------------------CLASSIFIER 4 -----------------------------#
    acc_train_scores = []
    acc_val_scores = []
    acc_train = 0
    acc_val = 0
    acc_test = 0
    
    # Counter for patience
    it_patience = 0
    
    # Best classifier in early stopping
    best_clf = None
    it_stop = 0 
    
    scaler = StandardScaler()
    clf = MLPClassifier(hidden_layer_sizes=(10,8,5), activation='relu', solver='adam', alpha=0.0001,
                        learning_rate='constant', learning_rate_init=0.01,max_iter=max_iter)
    classes = np.unique(Y_train)

    for it_ in range(max_iter):
        # Scaler fit/transform
        scaler.partial_fit(X_train, Y_train)
        X_train_ = scaler.transform(X_train)
        X_val_ = scaler.transform(X_val)
        
        # Classifier fit / evaluate
        clf.partial_fit(X_train_,Y_train,classes=classes)
        acc_train_scores.append(clf.score(X_train_,Y_train))
        acc_val_scores.append(clf.score(X_val_,Y_val))
        
        if acc_val_scores[-1] >= acc_val:
            acc_val = acc_val_scores[-1]
            best_clf = deepcopy(clf)
            it_stop = it_
            it_patience = 0
        else: 
            it_patience += 1
        
        if it_patience >= patience:
            break
        
    X_test_ = scaler.transform(X_test)    
    acc_test = best_clf.score(X_test_,Y_test)
    acc_train = acc_train_scores[it_stop] 
     
    dic_aux['clf_4']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test,
                      'acc_train_scores': acc_train_scores, 'acc_val_scores': acc_val_scores, 'it_stop':it_stop}
    
    #-----------------------CLASSIFIER 5 -----------------------------#
    acc_train_scores = []
    acc_val_scores = []
    acc_train = 0
    acc_val = 0
    acc_test = 0
    
    # Counter for patience
    it_patience = 0
    
    # Best classifier in early stopping
    best_clf = None
    it_stop = 0 
    
    scaler = StandardScaler()
    clf = MLPClassifier(hidden_layer_sizes=(10,8,3), activation='relu', solver='adam', alpha=0.0001,
                        learning_rate='constant', learning_rate_init=0.01,max_iter=max_iter)
    classes = np.unique(Y_train)

    for it_ in range(max_iter):
        # Scaler fit/transform
        scaler.partial_fit(X_train, Y_train)
        X_train_ = scaler.transform(X_train)
        X_val_ = scaler.transform(X_val)
        
        # Classifier fit / evaluate
        clf.partial_fit(X_train_,Y_train,classes=classes)
        acc_train_scores.append(clf.score(X_train_,Y_train))
        acc_val_scores.append(clf.score(X_val_,Y_val))
        
        if acc_val_scores[-1] >= acc_val:
            acc_val = acc_val_scores[-1]
            best_clf = deepcopy(clf)
            it_stop = it_
            it_patience = 0
        else: 
            it_patience += 1
        
        if it_patience >= patience:
            break
        
    X_test_ = scaler.transform(X_test)    
    acc_test = best_clf.score(X_test_,Y_test)
    acc_train = acc_train_scores[it_stop] 
     
    dic_aux['clf_5']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test,
                      'acc_train_scores': acc_train_scores, 'acc_val_scores': acc_val_scores, 'it_stop':it_stop}
    
    #-----------------------CLASSIFIER 6 -----------------------------#
    acc_train_scores = []
    acc_val_scores = []
    acc_train = 0
    acc_val = 0
    acc_test = 0
    
    # Counter for patience
    it_patience = 0
    
    # Best classifier in early stopping
    best_clf = None
    it_stop = 0 
    
    scaler = StandardScaler()
    clf = MLPClassifier(hidden_layer_sizes=(10,4,2), activation='relu', solver='adam', alpha=0.0001,
                        learning_rate='constant', learning_rate_init=0.01,max_iter=max_iter)
    classes = np.unique(Y_train)

    for it_ in range(max_iter):
        # Scaler fit/transform
        scaler.partial_fit(X_train, Y_train)
        X_train_ = scaler.transform(X_train)
        X_val_ = scaler.transform(X_val)
        
        # Classifier fit / evaluate
        clf.partial_fit(X_train_,Y_train,classes=classes)
        acc_train_scores.append(clf.score(X_train_,Y_train))
        acc_val_scores.append(clf.score(X_val_,Y_val))
        
        if acc_val_scores[-1] >= acc_val:
            acc_val = acc_val_scores[-1]
            best_clf = deepcopy(clf)
            it_stop = it_
            it_patience = 0
        else: 
            it_patience += 1
        
        if it_patience >= patience:
            break
        
    X_test_ = scaler.transform(X_test)    
    acc_test = best_clf.score(X_test_,Y_test)
    acc_train = acc_train_scores[it_stop] 
     
    dic_aux['clf_6']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test,
                      'acc_train_scores': acc_train_scores, 'acc_val_scores': acc_val_scores, 'it_stop':it_stop}
    
    info_clfs[f'it_{it}']=dic_aux
    

In [22]:
#Generamos la tabla de los acc para cada iteración y clasificador
n_clfs = 6

matrix_acc_train = np.zeros((n_clfs,N_it))
matrix_acc_val = np.zeros((n_clfs,N_it))
matrix_acc_test = np.zeros((n_clfs,N_it))

for it0 in range(N_it):
    for it1 in range(n_clfs):
        matrix_acc_train[it1,it0] = info_clfs[f'it_{it0}'][f'clf_{it1+1}']['acc_train']
        matrix_acc_val[it1,it0] = info_clfs[f'it_{it0}'][f'clf_{it1+1}']['acc_val']
        matrix_acc_test[it1,it0] = info_clfs[f'it_{it0}'][f'clf_{it1+1}']['acc_test']

acc_train_clfs_mean = matrix_acc_train.mean(axis=1,keepdims=True)
acc_train_clfs_std = np.std(matrix_acc_train,axis=1,keepdims=True)

acc_val_clfs_mean = matrix_acc_val.mean(axis=1,keepdims=True)
acc_val_clfs_std = np.std(matrix_acc_val,axis=1,keepdims=True)

acc_test_clfs_mean = matrix_acc_test.mean(axis=1,keepdims=True)
acc_test_clfs_std = np.std(matrix_acc_test,axis=1,keepdims=True)

In [23]:
accs_train_df = pd.DataFrame(matrix_acc_train,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6'])
accs_val_df = pd.DataFrame(matrix_acc_val,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6'])
accs_test_df = pd.DataFrame(matrix_acc_test,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6'])
accs_test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
clf_1,0.618133,0.359813,0.571875,0.669426,0.396247,0.500267,0.450772,0.343683,0.518499,0.667878,0.547507,0.564718,0.516415,0.628104,0.384655,0.616967,0.376161,0.496410,0.579771,0.500000
clf_2,0.569067,0.344237,0.516667,0.617175,0.388204,0.474078,0.465141,0.352248,0.515856,0.728594,0.475591,0.582985,0.505472,0.563656,0.398652,0.605141,0.487100,0.515385,0.526590,0.512042
clf_3,0.491733,0.566978,0.565104,0.724263,0.387131,0.494923,0.493348,0.489829,0.491015,0.703684,0.609449,0.605428,0.498176,0.657686,0.434422,0.628792,0.543344,0.527692,0.615224,0.558115
clf_4,0.482667,0.495327,0.554688,0.753751,0.330295,0.454303,0.437467,0.518201,0.520613,0.695381,0.413648,0.521921,0.488796,0.630745,0.407983,0.574807,0.537152,0.516923,0.607404,0.551832
clf_5,0.493333,0.443406,0.581250,0.749095,0.383914,0.480492,0.453965,0.485546,0.531184,0.682927,0.490289,0.546451,0.455446,0.623877,0.440643,0.593316,0.535088,0.546154,0.462982,0.529319
clf_6,0.489067,0.525961,0.546354,0.753233,0.343164,0.468733,0.482704,0.491970,0.439218,0.688116,0.407349,0.498434,0.477332,0.569995,0.502851,0.571208,0.526832,0.533846,0.493743,0.503141


In [24]:
results_df = pd.DataFrame(np.concatenate((acc_train_clfs_mean,acc_train_clfs_std,
                                          acc_val_clfs_mean,acc_val_clfs_std,
                                          acc_test_clfs_mean,acc_test_clfs_std),axis=1),
                          columns = ['TRAIN MEAN', 'TRAIN STD','VAL MEAN', 'VAL STD','TEST MEAN', 'TEST STD']
                          ,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6'])
results_df

,TRAIN MEAN,TRAIN STD,VAL MEAN,VAL STD,TEST MEAN,TEST STD
clf_1,0.601106,0.026621,0.468946,0.067860,0.515365,0.100182
clf_2,0.595184,0.025863,0.457852,0.062509,0.507194,0.090795
clf_3,0.687994,0.020257,0.536922,0.087195,0.554317,0.084472
clf_4,0.676823,0.024937,0.573336,0.070929,0.524695,0.096006
clf_5,0.676532,0.027769,0.579028,0.076739,0.525434,0.085727
clf_6,0.628204,0.073832,0.578455,0.072354,0.515663,0.086075


In [18]:
from scipy.stats import ks_1samp
from scipy.stats import ttest_1samp
from scipy.stats import wilcoxon
from scipy import stats

In [25]:
alpha = 0.05
alpha_bonferroni = alpha / 6

test_norm = pd.DataFrame(columns=['statistics','p-value','H0'])
clf_names = accs_test_df.index.values.tolist()

for clf in clf_names: 
    acc_test = accs_test_df.loc[clf]
    norm_test = ks_1samp(acc_test,stats.norm.cdf)
    if norm_test.pvalue <= alpha_bonferroni:
        test_norm.loc[clf]=[norm_test.statistic,norm_test.pvalue,False]
    else:
        test_norm.loc[clf]=[norm_test.statistic,norm_test.pvalue,True]

print(f'Significancia: {alpha_bonferroni:.2}')
print(f'Intervalo de confianza: {100*(1-alpha_bonferroni)}')
test_norm

Significancia: 0.0083
Intervalo de confianza: 99.16666666666667


,statistics,p-value,H0
clf_1,0.634458,2.436731e-08,False
clf_2,0.634666,2.404184e-08,False
clf_3,0.650671,8.363923e-09,False
clf_4,0.629411,3.368148e-08,False
clf_5,0.649479,9.061847e-09,False
clf_6,0.634262,2.467661e-08,False


In [26]:
mu = 0.5
test = pd.DataFrame(columns=['statics','p-value','H0'])

for clf in clf_names: 
    acc_test = accs_test_df.loc[clf]
    w_test = wilcoxon(acc_test-mu)
    if w_test.pvalue <= alpha_bonferroni:
        test.loc[clf]=[w_test.statistic,w_test.pvalue,False]
    else: 
        test.loc[clf]=[w_test.statistic,w_test.pvalue,True]

print(f'Significancia: {alpha_bonferroni:.2}')
print(f'Intervalo de confianza: {100*(1-alpha_bonferroni)}')
test

Significancia: 0.0083
Intervalo de confianza: 99.16666666666667


/home/bruno/anaconda3/envs/fairness1.0/lib/python3.10/site-packages/scipy/stats/_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,statics,p-value,H0
clf_1,76.0,0.444509,True
clf_2,95.0,0.728506,True
clf_3,47.0,0.029575,True
clf_4,75.0,0.277355,True
clf_5,81.0,0.388376,True
clf_6,91.0,0.621513,True
